In [1]:
import pandas as pd
import re
from ast import literal_eval


In [2]:
train = pd.read_csv('./materials/민원22~24년도통합_train_1010_law.csv' ,encoding='utf-8-sig', lineterminator='\n')
train['추출된 법(re)'] = train['추출된 법(re)'].apply(literal_eval)
train = train[train['추출된 법(re)'].apply(len) != 0]
train = train.reset_index(drop=True)



In [3]:
passage = pd.read_csv('./law_data/law_filter_raw.csv', encoding='utf-8-sig', lineterminator='\n')


In [4]:
def split_law_and_article(text):
    # '제숫자조' 뒤에 의2와 같은 추가 내용을 포함하도록 정규식 수정
    match = re.match(r'(.*?)(제\d+조[\w]*)', text)
    if match:
        law_name = match.group(1).replace(' ', '').strip()  # 법 이름 부분
        article = match.group(2).strip()  # 조항 부분
        return [law_name, article]
    else:
        return [text.strip(), '']
    
def split_passage(text):
    """
    입력된 텍스트를 법 이름과 조항으로 분리.
    """
    parts = text.split(' : ', 1)
    if len(parts) < 2:
        return '', ''
    law_parts = parts[0].split(' ; ')
    if len(law_parts) == 3:
        return law_parts[0].strip(), ' '.join(law_parts[1:]).strip()
    elif len(law_parts) == 2:
        return law_parts[0].strip(), law_parts[1].strip()
    return '', ''
passage[['law_name', 'law_clause']] = passage['passage'].apply(lambda x: pd.Series(split_passage(x)))

law_name_list=list(set(passage['law_name'].tolist()))

In [5]:
train['추출된 법(split)'] = train['추출된 법(re)'].apply(lambda x: [split_law_and_article(law) for law in x])
train['추출된법_law_name'] = train['추출된 법(split)'].apply(lambda x: [law[0] for law in x])
with open('./minwon_law_list.txt', 'r', encoding='utf-8-sig') as f:
    minwon_law_list = [line.strip() for line in f.readlines()]
unique_laws = list(set(minwon_law_list) - set(law_name_list))
train_unique_minwon = []
train_unique_minwon_list = []
for i in range(len(train)):
    index_list = []
    for law in range(len(train['추출된법_law_name'][i])):
        if train['추출된법_law_name'][i][law] in unique_laws:
            index_list.append(law)
            train_unique_minwon.append(i)
    train_unique_minwon_list.append(index_list)
train_unique_minwon = list(set(train_unique_minwon))

In [9]:
train['추출된 법(re)'][11]

['건축법 제19조',
 '건축물의 분양에 관한 법률 제6조',
 '표시광고의 공정화에 관한 법률 제3조',
 '건축물의 분양에 관한 법률 시행령 제9조']

In [10]:
train['추출된 법(split)'][11]

[['건축법', '제19조'],
 ['건축물의분양에관한법률', '제6조'],
 ['표시광고의공정화에관한법률', '제3조'],
 ['건축물의분양에관한법률시행령', '제9조']]

In [11]:
train['추출된법_law_name'][11]

['건축법', '건축물의분양에관한법률', '표시광고의공정화에관한법률', '건축물의분양에관한법률시행령']

In [8]:
train_unique_minwon_list[11]

[2]

In [ ]:
'표시광고의공정화에관한법률' in law_name_list

True

In [14]:
for idx in range(len(train_unique_minwon_list)):
    if train_unique_minwon_list[idx] == 0:
        pass
    else:
        list_ = sorted(train_unique_minwon_list[idx], reverse=True)
        law_list = train['추출된 법(re)'][idx]
        for i in list_:
            del law_list[i]
        train.at[idx, '추출된 법(re)'] = law_list

In [15]:
train['추출된 법(re)'][11]

['건축법 제19조', '건축물의 분양에 관한 법률 제6조', '건축물의 분양에 관한 법률 시행령 제9조']

In [ ]:
train = train[train['추출된 법(re)'].apply(len) != 0]
train = train.reset_index(drop=True)
train = train.drop(columns = ['추출된 법(split)', '추출된법_law_name'])
train.to_csv('./materials/민원22~24년도통합_train_1010_partial_drop.csv', encoding='utf-8-sig')
